# **8.2 Querying and Merging**

In [1]:
import pandas as pd

weather = pd.read_csv('nyc_weather_2018.csv')
weather

,attributes,datatype,date,station,value
0,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1CTFR0039,0.0
1,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1NJBG0015,0.0
2,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NJBG0015,0.0
3,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1NJBG0017,0.0
4,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NJBG0017,0.0
...,...,...,...,...,...
80251,",,W,",WDF5,2018-12-31T00:00:00,GHCND:USW00094789,130.0
80252,",,W,",WSF2,2018-12-31T00:00:00,GHCND:USW00094789,9.8
80253,",,W,",WSF5,2018-12-31T00:00:00,GHCND:USW00094789,12.5
80254,",,W,",WT01,2018-12-31T00:00:00,GHCND:USW00094789,1.0


In [2]:
#Using query (similar to SQL 'exp == exp')

snow_data = weather.query('datatype == "SNOW" and value > 0')
snow_data.head() 

,attributes,datatype,date,station,value
124,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NYWC0019,25.0
723,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0015,229.0
726,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0017,10.0
730,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0018,46.0
737,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJES0018,10.0


In [3]:
#Using sqlite to see the comparison of the above code using sqlite query

import sqlite3

with sqlite3.connect('weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" and value > 0', connection
    )
snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

True

In [4]:
#Merging DataFrames

station_info= pd.read_csv('weather_stations.csv')
station_info.head()


,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.0641,-73.5770,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.0378,-73.5682,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.9213,-74.0020,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.9027,-74.0834,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.9147,-73.9775,21.6


In [5]:
weather.head()

,attributes,datatype,date,station,value
0,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1CTFR0039,0.0
1,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1NJBG0015,0.0
2,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NJBG0015,0.0
3,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1NJBG0017,0.0
4,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NJBG0017,0.0


In [6]:
"""
We use join() operator for two columns that are similar. In this case, we use the weather.station column
and station_info.id column. However, row counts are different hence its important to keep a clear visual 
on the shape of the overall data set.

"""

'\nWe use join() operator for two columns that are similar. In this case, we use the weather.station column\nand station_info.id column. However, row counts are different hence its important to keep a clear visual \non the shape of the overall data set.\n\n'

In [7]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]

get_row_count(station_info, weather)

[262, 80256]

In [8]:
#Using the getattr() and map() functions
def get_info(attr, *dfs):
    return list(map(lambda x : getattr(x,attr), dfs))
get_info('shape' , station_info, weather)

[(262, 5), (80256, 5)]

Merge Methods, their SQL counterparts,  and their descriptions

left = Left Outer Join = Uses Keys from left frame only
right = Right Outer Join = Uses keys from right frame only
outer = Full outer Join = Uses union of keys from both frames
inner = Inner Join = Uses intersection of keys from both frames (Default)
cross = Cross Join = Uses cartesian product of rows from both frames


In [9]:
#By default merge function uses inner_join

inner_join = weather.merge(station_info, left_on='station', right_on = 'id')
inner_join.sample(5, random_state=0) #Sample() returns a sample of items from a specific axis


,attributes,datatype,date,station,value,id,name,latitude,longitude,elevation
27422,",,W,",WDF5,2018-04-29T00:00:00,GHCND:USW00094741,310.0,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",40.85000,-74.06139,2.7
19317,",,W,",WSF5,2018-03-24T00:00:00,GHCND:USW00094728,8.5,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7
13778,",,W,",PGTM,2018-03-01T00:00:00,GHCND:USW00054743,2351.0,GHCND:USW00054743,"CALDWELL ESSEX CO AIRPORT, NJ US",40.87639,-74.28306,52.7
39633,",,W,",WSF2,2018-06-25T00:00:00,GHCND:USW00094789,11.2,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.63860,-73.76220,3.4
51025,",,7,0700",TMAX,2018-08-19T00:00:00,GHCND:USC00280907,30.0,GHCND:USC00280907,"BOONTON 1 SE, NJ US",40.89174,-74.39635,85.3


In [10]:
#Since station and id yields the same values, we could remove duplication using on (parameter passed from rename())
weather.merge(station_info.rename(dict(id = 'station'), axis = 1), on= 'station').sample(5, random_state=0)

,attributes,datatype,date,station,value,name,latitude,longitude,elevation
27422,",,W,",WDF5,2018-04-29T00:00:00,GHCND:USW00094741,310.0,"TETERBORO AIRPORT, NJ US",40.85000,-74.06139,2.7
19317,",,W,",WSF5,2018-03-24T00:00:00,GHCND:USW00094728,8.5,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7
13778,",,W,",PGTM,2018-03-01T00:00:00,GHCND:USW00054743,2351.0,"CALDWELL ESSEX CO AIRPORT, NJ US",40.87639,-74.28306,52.7
39633,",,W,",WSF2,2018-06-25T00:00:00,GHCND:USW00094789,11.2,"JFK INTERNATIONAL AIRPORT, NY US",40.63860,-73.76220,3.4
51025,",,7,0700",TMAX,2018-08-19T00:00:00,GHCND:USC00280907,30.0,"BOONTON 1 SE, NJ US",40.89174,-74.39635,85.3


In [11]:
#Since we dont want to lose the rows with weather observations, we use left and right join instead of outer
left_join = station_info.merge(weather, left_on='id', right_on= 'station', how = 'left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how = 'right')
right_join.tail()

,attributes,datatype,date,station,value,id,name,latitude,longitude,elevation
80404,",,W,",WDF5,2018-12-31T00:00:00,GHCND:USW00094789,130.0,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4
80405,",,W,",WSF2,2018-12-31T00:00:00,GHCND:USW00094789,9.8,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4
80406,",,W,",WSF5,2018-12-31T00:00:00,GHCND:USW00094789,12.5,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4
80407,",,W,",WT01,2018-12-31T00:00:00,GHCND:USW00094789,1.0,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4
80408,",,W,",WT02,2018-12-31T00:00:00,GHCND:USW00094789,1.0,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4


In [12]:
left_join.sort_index(axis=1).sort_values(['date','station']).reset_index().drop(columns='index').equals(
    right_join.sort_index(axis=1).sort_values(['date', 'station']).reset_index().drop(columns='index')
)

True

In [13]:
#Checking the shape again
get_info('shape', inner_join, left_join, right_join)

[(80256, 10), (80409, 10), (80409, 10)]

In [14]:
outer_join = weather.merge(
    station_info[station_info.name.str.contains('NY')],
    left_on='station', right_on='id', how='outer', indicator=True
)


In [15]:
outer_join

,attributes,datatype,date,station,value,id,name,latitude,longitude,elevation,_merge
0,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1CTFR0039,0.0,NaN,NaN,NaN,NaN,NaN,left_only
1,",,N,",PRCP,2018-01-02T00:00:00,GHCND:US1CTFR0039,0.0,NaN,NaN,NaN,NaN,NaN,left_only
2,",,N,",PRCP,2018-01-03T00:00:00,GHCND:US1CTFR0039,0.0,NaN,NaN,NaN,NaN,NaN,left_only
3,",,N,",DAPR,2018-01-05T00:00:00,GHCND:US1CTFR0039,2.0,NaN,NaN,NaN,NaN,NaN,left_only
4,",,N,",MDPR,2018-01-05T00:00:00,GHCND:US1CTFR0039,15.5,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
80315,",,W,",WDF5,2018-12-31T00:00:00,GHCND:USW00094789,130.0,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,both
80316,",,W,",WSF2,2018-12-31T00:00:00,GHCND:USW00094789,9.8,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,both
80317,",,W,",WSF5,2018-12-31T00:00:00,GHCND:USW00094789,12.5,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,both
80318,",,W,",WT01,2018-12-31T00:00:00,GHCND:USW00094789,1.0,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,both


In [16]:
#Using inner_join with sqlite 
with sqlite3.connect('weather.db') as connection:
    inner_join_from_db = pd.read_sql(
    'SELECT * FROM weather JOIN stations ON weather.station == stations.id',
    connection
    )

inner_join_from_db.shape == inner_join.shape

True

# **Using Dirty Data**

In [17]:
#Using Dirty Data

dirty_data = pd.read_csv('dirty_data2.csv', index_col='date').drop_duplicates().drop(columns = 'SNWD').head()
dirty_data


,station,PRCP,SNOW,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01T00:00:00,?,0.0,0.0,5505.0,-40.0,NaN,NaN,NaN
2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-8.3,-16.1,-12.2,NaN,False
2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-4.4,-13.9,-13.3,NaN,False
2018-01-04T00:00:00,?,20.6,229.0,5505.0,-40.0,NaN,19.3,True
2018-01-05T00:00:00,?,0.3,NaN,5505.0,-40.0,NaN,NaN,NaN


In [18]:
#Dropping unecessary columns
valid_station = dirty_data.query('station != "?"').copy().drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query('station == "?"').copy().drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

In [19]:
valid_station

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather
date,,,,,,
2018-01-02T00:00:00,0.0,0.0,-8.3,-16.1,-12.2,False
2018-01-03T00:00:00,0.0,0.0,-4.4,-13.9,-13.3,False


In [20]:
station_with_wesf

,PRCP,SNOW,WESF,inclement_weather
date,,,,
2018-01-01T00:00:00,0.0,0.0,NaN,NaN
2018-01-04T00:00:00,20.6,229.0,19.3,True
2018-01-05T00:00:00,0.3,NaN,NaN,NaN


In [21]:
#Using join() function with left_index and right_index
valid_station.merge(
    station_with_wesf, left_index=True, right_index=True
).query('WESF > 0').head()

,PRCP_x,SNOW_x,TMAX,TMIN,TOBS,inclement_weather_x,PRCP_y,SNOW_y,WESF,inclement_weather_y
date,,,,,,,,,,


In [22]:
#Cant seem to find a reason on why this dataFrame is empty
valid_station.join(station_with_wesf, rsuffix='_?').query('WESF > 0').head() 

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,


Continuing with the Procedure

In [23]:
weather.set_index('station' , inplace=True) #Sets the index in weather as station permanently
station_info.set_index('id', inplace= True) #Same procedure but with different df and key


In [24]:
#using .intersection() function to to see the similar indexes
weather.index.intersection(station_info.index)

Index(['GHCND:US1CTFR0039', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017',
       'GHCND:US1NJBG0018', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0030',
       'GHCND:US1NJBG0039', 'GHCND:US1NJBG0044', 'GHCND:US1NJES0018',
       'GHCND:US1NJES0024',
       ...
       'GHCND:US1NJMS0047', 'GHCND:US1NYSF0083', 'GHCND:US1NYNY0074',
       'GHCND:US1NJPS0018', 'GHCND:US1NJBG0037', 'GHCND:USC00284987',
       'GHCND:US1NJES0031', 'GHCND:US1NJMD0086', 'GHCND:US1NJMS0097',
       'GHCND:US1NJMN0081'],
      dtype='object', length=109)

In [25]:
#Using the difference function to see which rows are lost from merging
weather.index.difference(station_info.index)

Index([], dtype='object')

In [26]:
#Same procedure but with station_info table
station_info.index.difference(weather.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=153)

In [28]:
ny_in_name = station_info[station_info.name.str.contains('NY')]

ny_in_name.index.difference(weather.index).shape[0]\
+ weather.index.difference(ny_in_name.index).shape[0]\
== weather.index.symmetric_difference(ny_in_name).shape[0]

False

In [29]:
weather.index.unique().union(station_info.index)


Index(['GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001',
       'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005',
       'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010',
       'GHCND:US1NJBG0011',
       ...
       'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734',
       'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787',
       'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745',
       'GHCND:USW00094789'],
      dtype='object', length=262)

In [30]:
ny_in_name = station_info[station_info.name.str.contains['NY']]
ny_in_name.index.difference(weather.index).union(weather.index(ny_in_name.index)).equals(
    weather.index.symmetric_difference(ny_in_name.index)
)

TypeError: 'method' object is not subscriptable